### 파이썬 미니 프로젝트 11조

In [50]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

#### 1. 데이터 수집

In [51]:
# url과 파라미터 설정 + 결과 확인
url = 'http://apis.data.go.kr/1352000/ODMS_STAT_09/callStat09Api'
key = os.getenv("SERVICE_KEY")
params ={'serviceKey' : f'{key}', 'apiType' : 'JSON'}

response = requests.get(url, params=params)
print(response.content)

# 서버의 응답 상태 출력
print(response.status_code) 

b'{"pageNo":"1","resultCode":"00","totalCount":68,"items":[{"dsStdrSrvrt":"13.6","allSvr":"14","allStdrSrvrt":"9.3","year":"2016","dsSvr":"11","othdsStdrSrvrt":"6","dsSrvrt":"13.9","othdsSvr":"3","dvsd":"\xec\x84\xb8\xec\xa2\x85","allSrvrt":"11.9","othdsSrvrt":"7.9"},{"dsStdrSrvrt":"8.6","allSvr":"71","allStdrSrvrt":"5.5","year":"2016","dsSvr":"59","othdsStdrSrvrt":"2.8","dsSrvrt":"5.5","othdsSvr":"12","dvsd":"\xec\xa0\x84\xeb\x82\xa8","allSrvrt":"4.7","othdsSrvrt":"2.8"},{"dsStdrSrvrt":"10.8","allSvr":"37","allStdrSrvrt":"7.8","year":"2017","dsSvr":"30","othdsStdrSrvrt":"5.1","dsSrvrt":"6","othdsSvr":"7","dvsd":"\xec\xa0\x9c\xec\xa3\xbc","allSrvrt":"5.8","othdsSrvrt":"5.5"},{"dsStdrSrvrt":"11.1","allSvr":"92","allStdrSrvrt":"8.1","year":"2016","dsSvr":"86","othdsStdrSrvrt":"2.3","dsSrvrt":"8.5","othdsSvr":"6","dvsd":"\xeb\x8c\x80\xea\xb5\xac","allSrvrt":"7","othdsSrvrt":"2.1"},{"dsStdrSrvrt":"8.8","allSvr":"69","allStdrSrvrt":"6.2","year":"2016","dsSvr":"59","othdsStdrSrvrt":"2.4","ds

In [52]:
def api_df(num):
    
    url = f'http://apis.data.go.kr/1352000/ODMS_STAT_{num}/callStat{num}Api'
    key = os.getenv("SERVICE_KEY")
    params ={'serviceKey' : f'{key}', 
             'apiType' : 'JSON'}

    response = requests.get(url, params=params)
    # 응답 상태가 정상인 경우 (200)에 데이터프레임 반환
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        return df
    else:
        print(f"오류 발생: {response.status_code}")

In [53]:
# 급성심장정지 발생률
heart_occur_eng = api_df('08')

# 급성심장정지 생존율
heart_live_eng = api_df('09')

# 급성심장정지 뇌기능 회복률
heart_heal_eng = api_df('10')

In [54]:
# 전국 병원 데이터 csv 파일 불러오기
hospital_df_eng = pd.read_csv("보건복지부_병원 및 의원 수_의료기관 종류별_시도별_20221231.csv", encoding='cp949')

#### 2. 데이터 전처리

In [55]:
# 급성심장정지 데이터프레임의 연도, 지역 컬럼을 앞으로 불러오기
# 알아보기 힘든 컬럼 이름 수정

front_cols = ['year', 'dvsd']
heart_occur_eng = heart_occur_eng[front_cols + [col for col in heart_occur_eng.columns if col not in front_cols]]

heart_occur = heart_occur_eng.rename(columns={
    'year': '연도',
    'dvsd': '시도',
    'allStdrOcrt': '전체 표준화 발생률',
    'othdsOcrt': '질병외 발생률',
    'dsStdrOcrt': '질병 표준화 발생률',
    'allOcrt': '전체 발생',
    'allOccr': '전체 발생률',
    'dsOcrt': '질병 발생률',
    'dsOccr': '질병 발생',
    'othdsStdrOcrt': '질병외 표준화 발생률',
    'othdsOccr': '질병외 발생'
})

ordered_occur = [
    '연도', '시도',
    '전체 발생', '전체 발생률', '전체 표준화 발생률',
    '질병 발생', '질병 발생률', '질병 표준화 발생률',
    '질병외 발생', '질병외 발생률', '질병외 표준화 발생률'
]

heart_occur = heart_occur[ordered_occur]
heart_occur = heart_occur.sort_values(by='연도').reset_index(drop=True)

heart_occur.head()

,연도,시도,전체 발생,전체 발생률,전체 표준화 발생률,질병 발생,질병 발생률,질병 표준화 발생률,질병외 발생,질병외 발생률,질병외 표준화 발생률
0,2016,전남,83.2,1575,46.9,1077,56.9,27.8,422,22.3,16.9
1,2016,대전,47,707,36.5,521,34.6,26.2,175,11.6,9.9
2,2016,경북,83.5,2241,49.3,1542,57.5,30.7,577,21.5,16
3,2016,세종,52.2,118,45.1,79,35,29.1,38,16.8,15.6
4,2016,광주,43.8,640,35.1,484,33.1,25.4,142,9.7,8.9


In [56]:
heart_live_eng = heart_live_eng[front_cols + [col for col in heart_live_eng.columns if col not in front_cols]]

heart_live = heart_live_eng.rename(columns={
    'year': '연도',
    'dvsd': '시도',
    'dsStdrSrvrt': '질병 표준화 생존률',
    'allSvr': '전체 생존',
    'allStdrSrvrt': '전체 표준화 생존률',
    'dsSvr': '질병 생존',
    'othdsStdrSrvrt': '질병외 표준화 생존률',
    'dsSrvrt': '질병 생존율',
    'othdsSvr': '질병외 생존',
    'allSrvrt': '전체 생존율',
    'othdsSrvrt': '질병외 생존율'
})

ordered_live = [
    '연도', '시도',
    '전체 생존', '전체 생존율', '전체 표준화 생존률',
    '질병 생존', '질병 생존율', '질병 표준화 생존률',
    '질병외 생존', '질병외 생존율', '질병외 표준화 생존률'
]

heart_live = heart_live[ordered_live]
heart_live = heart_live.sort_values(by='연도').reset_index(drop=True)

heart_live.head()

,연도,시도,전체 생존,전체 생존율,전체 표준화 생존률,질병 생존,질병 생존율,질병 표준화 생존률,질병외 생존,질병외 생존율,질병외 표준화 생존률
0,2016,세종,14,11.9,9.3,11,13.9,13.6,3,7.9,6
1,2016,부산,174,9.1,10.1,145,9.8,13.2,26,6.3,5.8
2,2016,대전,59,8.4,9.3,52,10,11.7,7,4,4
3,2016,경북,92,4.3,4.8,84,5.4,7.3,8,1.4,1.4
4,2016,인천,125,8.2,8.9,110,9.6,11.6,15,4,4.5


In [57]:
heart_heal_eng = heart_heal_eng[front_cols + [col for col in heart_heal_eng.columns if col not in front_cols]]

heart_heal = heart_heal_eng.rename(columns={
    'year': '연도',
    'dvsd': '시도',
    'allStbfnRcvr': '전체 표준화 뇌기능 회복률',
    'othdsStbfnRcvr': '질병외 표준화 뇌기능 회복률',
    'dsBrfr': '질병 뇌기능 회복',
    'allBrfr': '전체 뇌기능 회복',
    'dsBrfrr': '질병 뇌기능 회복률',
    'allBrfcRcvr': '전체 뇌기능 회복률',
    'dsStbfnRcvr': '질병 표준화 뇌기능 회복률',
    'othdsBrfr': '질병외 뇌기능 회복',
    'othdsBrfcRcvr': '질병외 뇌기능 회복률'
})

ordered_heal = [
    '연도', '시도',
    '전체 뇌기능 회복', '전체 뇌기능 회복률', '전체 표준화 뇌기능 회복률',
    '질병 뇌기능 회복', '질병 뇌기능 회복률', '질병 표준화 뇌기능 회복률',
    '질병외 뇌기능 회복', '질병외 뇌기능 회복률', '질병외 표준화 뇌기능 회복률'
]

heart_heal = heart_heal[ordered_heal]
heart_heal = heart_heal.sort_values(by='연도').reset_index(drop=True)

heart_heal.head()

,연도,시도,전체 뇌기능 회복,전체 뇌기능 회복률,전체 표준화 뇌기능 회복률,질병 뇌기능 회복,질병 뇌기능 회복률,질병 표준화 뇌기능 회복률,질병외 뇌기능 회복,질병외 뇌기능 회복률,질병외 표준화 뇌기능 회복률
0,2016,제주,23,3.7,4.7,18,3.9,6.2,5,3.1,3.2
1,2016,부산,101,5.3,6,93,6.3,9.2,5,1.2,1.1
2,2016,전북,45,3.3,4,44,4.5,6.4,1,0.3,0.2
3,2016,전남,30,2,2.6,29,2.7,5.1,1,0.2,0.3
4,2016,대전,28,4,4.3,27,5.2,6.6,1,0.6,0.5


In [58]:
# 병원 데이터 필요한 컬럼만 남기기 (연도, 시도, 종합병원, 일반병원)
hospital_df = hospital_df_eng[['연도', '시도', '병의원_종합병원', '병의원_일반병원']]

# 컬럼이름 수정
hospital_df = hospital_df.rename(columns={
    '병의원_종합병원': '종합병원',
    '병의원_일반병원': '일반병원'
})

hospital_df.head()

In [59]:
# 시도 컬럼의 데이터 내용 수정 (ex. 서울 Seoul -> 서울)
hospital_df['시도'] = hospital_df['시도'].str[:2]

# 급성심장정지 데이터와 데이터 추출 연도 일치를 위해 2016-2019년도 데이터 추출
hospital_df[hospital_df['연도'].between(2016,2019)].reset_index(drop=True)

hospital_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_16860\2966804921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_df['시도'] = hospital_df['시도'].str[:2]


,연도,시도,병의원_종합병원,병의원_일반병원
0,2015,서울,56,217
1,2015,부산,28,130
2,2015,대구,12,114
3,2015,인천,19,55
4,2015,광주,22,75


#### 3. 데이터 분석 (EDA & 상관관계 분석)

#### 4. 시각화